In [1]:
%cd /content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2

/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN, Reshape, GRU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [3]:
def read_and_reshape_data(filepath):
    tps_df = pd.read_pickle(filepath)

    reshaped_tps_df = pd.DataFrame()
    reshaped_tps_df['TIME'] = tps_df.time.unique()
    for seg in tps_df.segmentID.unique():
        column = tps_df[tps_df['segmentID'] == seg][['time', 'TrafficIndex_GP']].drop_duplicates(subset=['time'])
        column.columns = ['TIME', str(seg)]
        reshaped_tps_df = reshaped_tps_df.join(column.set_index('TIME'), on='TIME')

    reshaped_tps_df = reshaped_tps_df.set_index('TIME')
    return reshaped_tps_df

In [4]:
reshaped_tps_df = read_and_reshape_data('/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/tps_df.pkl')


In [5]:
reshaped_tps_df.shape

(14551, 87)

In [6]:
def create_sequences(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(data):
            break
        seq_x, seq_y = data[i:end_ix, :], data[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def normalize_features(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

def create_prediction_sequences(data, n_steps_in):
    X = []
    for i in range(len(data) - n_steps_in + 1):
        seq_x = data[i:i + n_steps_in, :]
        X.append(seq_x)
    return np.array(X)

## LSTM

In [7]:
def build_lstm_model(n_steps_in, n_features, n_steps_out):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps_in, n_features)))
    model.add(Dense(n_steps_out * n_features))
    model.add(Reshape((n_steps_out, n_features)))
    model.compile(optimizer='adam', loss='mse')
    return model

In [8]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/tps_df.pkl'
full_data = read_and_reshape_data(filepath)

n_steps_in, n_steps_out = 36, 12
n_features = full_data.shape[1]
scaled_data, scaler = normalize_features(full_data.to_numpy())
X, y = create_sequences(scaled_data, n_steps_in, n_steps_out)
y = y.reshape((y.shape[0], n_steps_out, n_features))


train_size = int(len(X) * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:], y[train_size:]

X_train = np.nan_to_num(X_train, nan=np.nanmean(X_train))
y_train = np.nan_to_num(y_train, nan=np.nanmean(y_train))

X_val = np.nan_to_num(X_val, nan=np.nanmean(X_train))
y_val = np.nan_to_num(y_val, nan=np.nanmean(y_train))


model = build_lstm_model(n_steps_in, n_features, n_steps_out)

In [10]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=10)])

Epoch 1/10
363/363 [==============================] - 15s 37ms/step - loss: 0.0400 - val_loss: 0.0023
Epoch 2/10
363/363 [==============================] - 12s 34ms/step - loss: 0.0079 - val_loss: 0.0027
Epoch 3/10
363/363 [==============================] - 11s 30ms/step - loss: 0.0071 - val_loss: 0.0029
Epoch 4/10
363/363 [==============================] - 8s 21ms/step - loss: 0.0067 - val_loss: 0.0021
Epoch 5/10
363/363 [==============================] - 10s 28ms/step - loss: 0.0064 - val_loss: 0.0019
Epoch 6/10
363/363 [==============================] - 9s 24ms/step - loss: 0.0062 - val_loss: 0.0018
Epoch 7/10
363/363 [==============================] - 9s 24ms/step - loss: 0.0061 - val_loss: 0.0038
Epoch 8/10
363/363 [==============================] - 10s 27ms/step - loss: 0.0059 - val_loss: 0.0019
Epoch 9/10
363/363 [==============================] - 8s 21ms/step - loss: 0.0057 - val_loss: 0.0016
Epoch 10/10
363/363 [==============================] - 10s 27ms/step - loss: 0.0056 - 

## RNN

In [12]:
def build_rnn_model(n_steps_in, n_features, n_steps_out):
    model = Sequential()
    model.add(SimpleRNN(50, activation='relu', input_shape=(n_steps_in, n_features)))
    model.add(Dense(n_steps_out * n_features))
    model.add(Reshape((n_steps_out, n_features)))
    model.compile(optimizer='adam', loss='mse')
    return model

In [13]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/tps_df.pkl'
full_data = read_and_reshape_data(filepath)

n_steps_in, n_steps_out = 36, 12
n_features = full_data.shape[1]
scaled_data, scaler = normalize_features(full_data.to_numpy())
X, y = create_sequences(scaled_data, n_steps_in, n_steps_out)
y = y.reshape((y.shape[0], n_steps_out, n_features))


train_size = int(len(X) * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:], y[train_size:]

X_train = np.nan_to_num(X_train, nan=np.nanmean(X_train))
y_train = np.nan_to_num(y_train, nan=np.nanmean(y_train))

X_val = np.nan_to_num(X_val, nan=np.nanmean(X_train))
y_val = np.nan_to_num(y_val, nan=np.nanmean(y_train))


model = build_rnn_model(n_steps_in, n_features, n_steps_out)
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=10)])

Epoch 1/10
363/363 [==============================] - 6s 14ms/step - loss: 0.0470 - val_loss: 0.0024
Epoch 2/10
363/363 [==============================] - 6s 17ms/step - loss: 0.0090 - val_loss: 0.0028
Epoch 3/10
363/363 [==============================] - 5s 13ms/step - loss: 0.0081 - val_loss: 0.0027
Epoch 4/10
363/363 [==============================] - 5s 13ms/step - loss: 0.0077 - val_loss: 0.0023
Epoch 5/10
363/363 [==============================] - 6s 17ms/step - loss: 0.0074 - val_loss: 0.0024
Epoch 6/10
363/363 [==============================] - 5s 13ms/step - loss: 0.0071 - val_loss: 0.0024
Epoch 7/10
363/363 [==============================] - 5s 13ms/step - loss: 0.0070 - val_loss: 0.0023
Epoch 8/10
363/363 [==============================] - 6s 15ms/step - loss: 0.0068 - val_loss: 0.0022
Epoch 9/10
363/363 [==============================] - 4s 12ms/step - loss: 0.0067 - val_loss: 0.0026
Epoch 10/10
363/363 [==============================] - 6s 17ms/step - loss: 0.0065 - val_lo

## GRU

In [14]:
def build_gru_model(n_steps_in, n_features, n_steps_out):
    model = Sequential()
    model.add(GRU(50, activation='relu', input_shape=(n_steps_in, n_features)))
    model.add(Dense(n_steps_out * n_features))
    model.add(Reshape((n_steps_out, n_features)))
    model.compile(optimizer='adam', loss='mse')
    return model

In [15]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/tps_df.pkl'
full_data = read_and_reshape_data(filepath)

n_steps_in, n_steps_out = 36, 12
n_features = full_data.shape[1]
scaled_data, scaler = normalize_features(full_data.to_numpy())
X, y = create_sequences(scaled_data, n_steps_in, n_steps_out)
y = y.reshape((y.shape[0], n_steps_out, n_features))


train_size = int(len(X) * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:], y[train_size:]

X_train = np.nan_to_num(X_train, nan=np.nanmean(X_train))
y_train = np.nan_to_num(y_train, nan=np.nanmean(y_train))

X_val = np.nan_to_num(X_val, nan=np.nanmean(X_train))
y_val = np.nan_to_num(y_val, nan=np.nanmean(y_train))


model = build_gru_model(n_steps_in, n_features, n_steps_out)
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=10)])

Epoch 1/10
363/363 [==============================] - 12s 28ms/step - loss: 0.0400 - val_loss: 0.0019
Epoch 2/10
363/363 [==============================] - 10s 26ms/step - loss: 0.0090 - val_loss: 0.0019
Epoch 3/10
363/363 [==============================] - 9s 24ms/step - loss: 0.0074 - val_loss: 0.0019
Epoch 4/10
363/363 [==============================] - 10s 29ms/step - loss: 0.0069 - val_loss: 0.0019
Epoch 5/10
363/363 [==============================] - 10s 26ms/step - loss: 0.0066 - val_loss: 0.0022
Epoch 6/10
363/363 [==============================] - 8s 22ms/step - loss: 0.0064 - val_loss: 0.0018
Epoch 7/10
363/363 [==============================] - 10s 26ms/step - loss: 0.0062 - val_loss: 0.0017
Epoch 8/10
363/363 [==============================] - 9s 25ms/step - loss: 0.0061 - val_loss: 0.0018
Epoch 9/10
363/363 [==============================] - 10s 28ms/step - loss: 0.0059 - val_loss: 0.0018
Epoch 10/10
363/363 [==============================] - 13s 37ms/step - loss: 0.0058 -

### Submission

In [64]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_1.pkl'
test_data = read_and_reshape_data(filepath)

test_data_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_rescaled = scaler.inverse_transform(predictions.reshape(-1, predictions.shape[2])).reshape(predictions.shape)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])
predictions_2d.shape

1/1 [==============================] - 0s 28ms/step


(12, 87)

In [65]:
test_data.tail()

,11,12,13,14,15,16,17,18,19,20,...,94,95,96,97,99,100,102,103,104,106
TIME,,,,,,,,,,,,,,,,,,,,,
2020-06-02 05:00:00.000,0.993,0.999,0.963,0.979,0.991,0.957,0.989,1.000,1.0,1.00,...,1.0,1.0,1.0,1.0,1.000,1.0,1.0,1.0,1.0,1.0
2020-06-02 05:15:00.000,0.994,0.996,0.931,0.960,0.977,0.932,0.989,1.000,1.0,1.00,...,1.0,1.0,1.0,1.0,1.000,1.0,1.0,1.0,1.0,1.0
2020-06-02 05:30:00.000,0.991,0.976,0.895,0.922,0.920,0.856,0.934,1.000,1.0,1.00,...,1.0,1.0,1.0,1.0,1.000,1.0,1.0,1.0,1.0,1.0
2020-06-02 05:45:00.000,0.987,0.985,0.905,0.926,0.709,0.806,0.737,1.000,1.0,1.00,...,1.0,1.0,1.0,1.0,1.000,1.0,1.0,1.0,1.0,1.0
2020-06-02 06:00:00.000,0.978,0.976,0.893,0.914,0.533,0.816,0.840,0.995,1.0,0.99,...,1.0,1.0,1.0,1.0,0.991,1.0,1.0,1.0,1.0,1.0


In [66]:
output_1 = pd.DataFrame(predictions_2d)
output_1.head()

,0,1,2,3,4,5,6,7,8,9,...,77,78,79,80,81,82,83,84,85,86
0,0.951181,0.957619,0.901025,0.887629,0.741718,0.665537,0.751927,0.943319,0.959233,0.940632,...,1.056597,0.991529,1.037860,1.026433,0.984263,1.034996,0.987427,0.985285,1.003949,1.010245
1,0.950984,0.936541,0.891965,0.873420,0.730579,0.655796,0.735285,0.926630,0.956911,0.953265,...,1.082736,0.951523,1.031487,0.994170,0.986147,0.983725,1.002935,1.001485,0.976203,0.993725
2,0.925605,0.941150,0.881808,0.857526,0.707629,0.666945,0.747093,0.923251,0.939382,0.957015,...,1.047952,0.941293,1.005366,0.998981,0.987514,0.987447,0.993837,1.018359,0.994565,1.010335
3,0.909781,0.957405,0.875170,0.822097,0.714337,0.637221,0.747832,0.884746,0.964005,0.934426,...,1.040375,0.919006,0.959322,1.006516,0.993861,0.985973,0.993478,0.988459,0.982194,0.989735
4,0.920912,0.950535,0.870194,0.819606,0.720669,0.645782,0.730061,0.899574,0.971398,0.928592,...,1.000422,0.907055,0.958502,1.004843,0.989105,0.963139,1.001089,0.994534,0.994555,1.000773


In [67]:
output_1.index = pd.date_range(start='2020-06-02 06:15:00', end='2020-06-02 09:00:00', freq='15min').astype(int) / 10**9
output_1.columns = reshaped_tps_df.columns

In [68]:
output_1.head()

,11,12,13,14,15,16,17,18,19,20,...,94,95,96,97,99,100,102,103,104,106
1.591078e+09,0.951181,0.957619,0.901025,0.887629,0.741718,0.665537,0.751927,0.943319,0.959233,0.940632,...,1.056597,0.991529,1.037860,1.026433,0.984263,1.034996,0.987427,0.985285,1.003949,1.010245
1.591079e+09,0.950984,0.936541,0.891965,0.873420,0.730579,0.655796,0.735285,0.926630,0.956911,0.953265,...,1.082736,0.951523,1.031487,0.994170,0.986147,0.983725,1.002935,1.001485,0.976203,0.993725
1.591080e+09,0.925605,0.941150,0.881808,0.857526,0.707629,0.666945,0.747093,0.923251,0.939382,0.957015,...,1.047952,0.941293,1.005366,0.998981,0.987514,0.987447,0.993837,1.018359,0.994565,1.010335
1.591081e+09,0.909781,0.957405,0.875170,0.822097,0.714337,0.637221,0.747832,0.884746,0.964005,0.934426,...,1.040375,0.919006,0.959322,1.006516,0.993861,0.985973,0.993478,0.988459,0.982194,0.989735
1.591082e+09,0.920912,0.950535,0.870194,0.819606,0.720669,0.645782,0.730061,0.899574,0.971398,0.928592,...,1.000422,0.907055,0.958502,1.004843,0.989105,0.963139,1.001089,0.994534,0.994555,1.000773


In [69]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_2.pkl'
test_data2 = read_and_reshape_data(filepath)
test_data2_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data2_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data2_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])
predictions_2d.shape

1/1 [==============================] - 0s 48ms/step


(12, 87)

In [70]:
test_data2.tail()

,11,12,13,14,15,16,17,18,19,20,...,94,95,96,97,99,100,102,103,104,106
TIME,,,,,,,,,,,,,,,,,,,,,
2020-06-03 06:00:00.000,0.974,0.978,0.892,0.910,0.627,0.775,0.664,0.958,1.000,1.000,...,1.0,1.000,1.0,1.0,0.989,1.0,1.0,1.0,1.0,1.0
2020-06-03 06:15:00.000,0.955,0.977,0.903,0.909,0.650,0.562,0.649,0.957,0.992,1.000,...,1.0,0.997,1.0,1.0,0.996,1.0,1.0,1.0,1.0,1.0
2020-06-03 06:30:00.000,0.943,0.955,0.866,0.870,0.413,0.528,0.570,0.945,0.990,1.000,...,1.0,0.993,1.0,1.0,0.993,1.0,1.0,1.0,1.0,1.0
2020-06-03 06:45:00.000,0.932,0.942,0.854,0.888,0.442,0.474,0.566,0.946,0.984,0.989,...,1.0,0.988,1.0,1.0,0.991,1.0,1.0,1.0,1.0,1.0
2020-06-03 07:00:00.000,0.950,0.924,0.854,0.885,0.767,0.502,0.562,0.922,0.979,0.996,...,1.0,0.986,1.0,1.0,0.979,1.0,1.0,1.0,1.0,1.0


In [71]:
output_2 = pd.DataFrame(predictions_2d)
output_2.index = pd.date_range(start='2020-06-03 07:15:00', end='2020-06-03 10:00:00', freq='15min').astype(int) / 10**9
output_2.columns = reshaped_tps_df.columns
print(output_2.shape)
output_2.head()

(12, 87)


,11,12,13,14,15,16,17,18,19,20,...,94,95,96,97,99,100,102,103,104,106
1.591168e+09,0.951181,0.957619,0.901025,0.887629,0.741718,0.665537,0.751927,0.943319,0.959233,0.940632,...,1.056597,0.991529,1.037860,1.026433,0.984263,1.034996,0.987427,0.985285,1.003949,1.010245
1.591169e+09,0.950984,0.936541,0.891965,0.873420,0.730579,0.655796,0.735285,0.926630,0.956911,0.953265,...,1.082736,0.951523,1.031487,0.994170,0.986147,0.983725,1.002935,1.001485,0.976203,0.993725
1.591170e+09,0.925605,0.941150,0.881808,0.857526,0.707629,0.666945,0.747093,0.923251,0.939382,0.957015,...,1.047952,0.941293,1.005366,0.998981,0.987514,0.987447,0.993837,1.018359,0.994565,1.010335
1.591171e+09,0.909781,0.957405,0.875170,0.822097,0.714337,0.637221,0.747832,0.884746,0.964005,0.934426,...,1.040375,0.919006,0.959322,1.006516,0.993861,0.985973,0.993478,0.988459,0.982194,0.989735
1.591172e+09,0.920912,0.950535,0.870194,0.819606,0.720669,0.645782,0.730061,0.899574,0.971398,0.928592,...,1.000422,0.907055,0.958502,1.004843,0.989105,0.963139,1.001089,0.994534,0.994555,1.000773


In [72]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_3.pkl'
test_data3 = read_and_reshape_data(filepath)
test_data3_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data3_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data3_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])
print(predictions_2d.shape)
test_data3.tail()

1/1 [==============================] - 0s 51ms/step
(12, 87)


,11,12,13,14,15,16,17,18,19,20,...,94,95,96,97,99,100,102,103,104,106
TIME,,,,,,,,,,,,,,,,,,,,,
2020-06-04 07:00:00.000,0.913,0.940,0.857,0.875,0.776,0.663,0.596,0.901,0.961,0.984,...,1.0,0.983,1.000,1.000,0.990,1.0,1.0,1.0,1.000,1.0
2020-06-04 07:15:00.000,0.942,0.944,0.876,0.904,0.847,0.786,0.620,0.932,0.978,0.979,...,1.0,0.970,1.000,0.994,0.982,1.0,1.0,1.0,1.000,1.0
2020-06-04 07:30:00.000,0.957,0.964,0.864,0.892,0.711,0.696,0.611,0.860,0.945,0.962,...,1.0,0.951,0.990,0.992,0.978,1.0,1.0,1.0,1.000,1.0
2020-06-04 07:45:00.000,0.935,0.952,0.884,0.920,0.532,0.547,0.554,0.855,0.953,0.952,...,1.0,0.971,0.974,1.000,0.995,1.0,1.0,1.0,0.987,1.0
2020-06-04 08:00:00.000,0.967,0.958,0.892,0.941,0.823,0.633,0.631,0.888,0.975,0.977,...,1.0,0.950,1.000,1.000,0.979,1.0,1.0,1.0,1.000,1.0


In [73]:
output_3 = pd.DataFrame(predictions_2d)
output_3.index = pd.date_range(start='2020-06-04 08:15:00', end='2020-06-04 11:00:00', freq='15min').astype(int) / 10**9
output_3.columns = reshaped_tps_df.columns
print(output_3.shape)
output_3.head()

(12, 87)


,11,12,13,14,15,16,17,18,19,20,...,94,95,96,97,99,100,102,103,104,106
1.591258e+09,0.951181,0.957619,0.901025,0.887629,0.741718,0.665537,0.751927,0.943319,0.959233,0.940632,...,1.056597,0.991529,1.037860,1.026433,0.984263,1.034996,0.987427,0.985285,1.003949,1.010245
1.591259e+09,0.950984,0.936541,0.891965,0.873420,0.730579,0.655796,0.735285,0.926630,0.956911,0.953265,...,1.082736,0.951523,1.031487,0.994170,0.986147,0.983725,1.002935,1.001485,0.976203,0.993725
1.591260e+09,0.925605,0.941150,0.881808,0.857526,0.707629,0.666945,0.747093,0.923251,0.939382,0.957015,...,1.047952,0.941293,1.005366,0.998981,0.987514,0.987447,0.993837,1.018359,0.994565,1.010335
1.591261e+09,0.909781,0.957405,0.875170,0.822097,0.714337,0.637221,0.747832,0.884746,0.964005,0.934426,...,1.040375,0.919006,0.959322,1.006516,0.993861,0.985973,0.993478,0.988459,0.982194,0.989735
1.591262e+09,0.920912,0.950535,0.870194,0.819606,0.720669,0.645782,0.730061,0.899574,0.971398,0.928592,...,1.000422,0.907055,0.958502,1.004843,0.989105,0.963139,1.001089,0.994534,0.994555,1.000773


In [74]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_4.pkl'
test_data4 = read_and_reshape_data(filepath)
test_data4_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data4_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data4_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])
print(predictions_2d.shape)

output_4 = pd.DataFrame(predictions_2d)
output_4.index = pd.date_range(start='2020-06-05 09:15:00', end='2020-06-05 12:00:00', freq='15min').astype(int) / 10**9
output_4.columns = reshaped_tps_df.columns
print(output_4.shape)


1/1 [==============================] - 0s 39ms/step
(12, 87)
(12, 87)


In [75]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_5.pkl'
test_data5 = read_and_reshape_data(filepath)
test_data5_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data5_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data5_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])


output_5 = pd.DataFrame(predictions_2d)
output_5.index = pd.date_range(start='2020-06-06 10:15:00', end='2020-06-06 13:00:00', freq='15min').astype(int) / 10**9
output_5.columns = reshaped_tps_df.columns
print(output_5.shape)


1/1 [==============================] - 0s 45ms/step
(12, 87)


In [76]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_6.pkl'
test_data6 = read_and_reshape_data(filepath)
test_data6_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data6_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data6_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])

output_6 = pd.DataFrame(predictions_2d)
output_6.index = pd.date_range(start='2020-06-07 11:15:00', end='2020-06-07 14:00:00', freq='15min').astype(int) / 10**9
output_6.columns = reshaped_tps_df.columns
print(output_6.shape)

1/1 [==============================] - 0s 43ms/step
(12, 87)


In [77]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_7.pkl'
test_data7 = read_and_reshape_data(filepath)
test_data7_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data7_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data7_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])

output_7 = pd.DataFrame(predictions_2d)
output_7.index = pd.date_range(start='2020-06-08 12:15:00', end='2020-06-08 15:00:00', freq='15min').astype(int) / 10**9
output_7.columns = reshaped_tps_df.columns
print(output_7.shape)

1/1 [==============================] - 0s 50ms/step
(12, 87)


In [78]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_8.pkl'
test_data8 = read_and_reshape_data(filepath)
test_data8_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data8_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data8_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])

output_8 = pd.DataFrame(predictions_2d)
output_8.index = pd.date_range(start='2020-06-09 13:15:00', end='2020-06-09 16:00:00', freq='15min').astype(int) / 10**9
output_8.columns = reshaped_tps_df.columns
print(output_8.shape)

1/1 [==============================] - 0s 28ms/step
(12, 87)


In [79]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_9.pkl'
test_data9 = read_and_reshape_data(filepath)
test_data9_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data9_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data9_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])

output_9 = pd.DataFrame(predictions_2d)
output_9.index = pd.date_range(start='2020-06-10 14:15:00', end='2020-06-10 17:00:00', freq='15min').astype(int) / 10**9
output_9.columns = reshaped_tps_df.columns
print(output_9.shape)

1/1 [==============================] - 0s 32ms/step
(12, 87)


In [80]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_10.pkl'
test_data10 = read_and_reshape_data(filepath)
test_data10_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data10_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data10_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])

output_10 = pd.DataFrame(predictions_2d)
output_10.index = pd.date_range(start='2020-06-11 15:15:00', end='2020-06-11 18:00:00', freq='15min').astype(int) / 10**9
output_10.columns = reshaped_tps_df.columns
print(output_10.shape)

1/1 [==============================] - 0s 28ms/step
(12, 87)


In [81]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_11.pkl'
test_data11 = read_and_reshape_data(filepath)
test_data11_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data11_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data11_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])

output_11 = pd.DataFrame(predictions_2d)
output_11.index = pd.date_range(start='2020-06-12 16:15:00', end='2020-06-12 19:00:00', freq='15min').astype(int) / 10**9
output_11.columns = reshaped_tps_df.columns
print(output_11.shape)

1/1 [==============================] - 0s 28ms/step
(12, 87)


In [82]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_12.pkl'
test_data12 = read_and_reshape_data(filepath)
test_data12_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data12_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data12_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])

output_12 = pd.DataFrame(predictions_2d)
output_12.index = pd.date_range(start='2020-06-13 17:15:00', end='2020-06-13 20:00:00', freq='15min').astype(int) / 10**9
output_12.columns = reshaped_tps_df.columns
print(output_12.shape)

1/1 [==============================] - 0s 34ms/step
(12, 87)


In [83]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_13.pkl'
test_data13 = read_and_reshape_data(filepath)
test_data13_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data13_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data13_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])

output_13 = pd.DataFrame(predictions_2d)
output_13.index = pd.date_range(start='2020-06-14 18:15:00', end='2020-06-14 21:00:00', freq='15min').astype(int) / 10**9
output_13.columns = reshaped_tps_df.columns
print(output_13.shape)

1/1 [==============================] - 0s 31ms/step
(12, 87)


In [84]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_14.pkl'
test_data14 = read_and_reshape_data(filepath)
test_data14_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data14_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data14_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])

output_14 = pd.DataFrame(predictions_2d)
output_14.index = pd.date_range(start='2020-06-15 19:15:00', end='2020-06-15 22:00:00', freq='15min').astype(int) / 10**9
output_14.columns = reshaped_tps_df.columns
print(output_14.shape)

1/1 [==============================] - 0s 30ms/step
(12, 87)


In [85]:
filepath = '/content/drive/MyDrive/CV_ENG_8160/Homeworks/HW2/Test/tps_15.pkl'
test_data15 = read_and_reshape_data(filepath)
test_data15_scaled = scaler.transform(test_data.to_numpy())

X_test, _ = create_sequences(test_data15_scaled, n_steps_in, n_steps_out)
X_test = create_prediction_sequences(test_data15_scaled, n_steps_in)

predictions = model.predict(X_test)
predictions_2d = predictions.reshape(predictions.shape[1], predictions.shape[2])

output_15 = pd.DataFrame(predictions_2d)
output_15.index = pd.date_range(start='2020-06-16 20:15:00', end='2020-06-16 23:00:00', freq='15min').astype(int) / 10**9
output_15.columns = reshaped_tps_df.columns
print(output_15.shape)

1/1 [==============================] - 0s 25ms/step
(12, 87)


In [86]:
final_output = pd.concat([output_1, output_2, output_3, output_4, output_5, output_6, output_7, output_8, output_9, output_10, output_11, output_12, output_13, output_14, output_15])
final_output.to_json('prediction_result_10.json')
